In [1]:
#!/usr/bin/python3
import os
import numpy as np
import string
import pandas as pd
from pandas import DataFrame
import random
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
import math
from collections import defaultdict
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from random import choice

In [2]:
#This routine returns all the paths from the node v
def DFS(G,v,seen=None,path=None):
    if seen is None: seen = []
    if path is None: path = [v]

    seen.append(v)

    paths = []
    for t in G[v]:
        if t not in seen:
            t_path = path + [t]
            paths.append(tuple(t_path))
            paths.extend(DFS(G, t, seen[:], t_path))
    return paths

#Given a set of paths this routine returns the maximum length chain
def give_a_max_path(all_paths):
        
    
    max_len   = max(len(p) for p in all_paths)
    
    max_paths = [p for p in all_paths if len(p) == max_len]
        
    num_max_paths = len(max_paths)
        
    chosen_path = max_paths[np.random.randint(num_max_paths)]
    
    return chosen_path, max_len

def union(list1,list2):
    return list(set(list1) | set(list2))



In [3]:
# Defining the tree dictionary for each node
# Each block tree is a dictionary
# Each node is in nodes[] list
# \ell fixed to 2

num_nodes = 10000

max_blocks = 100

#Chyeck list of lists

delay_vector= np.arange(0,2.25,0.25)

num_delays=len(delay_vector)

num_trials = 9

max_len_tracker = np.zeros((num_delays, max_blocks+1, num_trials))

latest_k=3

In [4]:
def latest_union(list1,list2,time):
    temp_list = list2
    for blk in list2:
        if int(blk)<(time-latest_k):
            temp_list.remove(blk)
    return list(set(list1) | set(temp_list))

In [5]:
for d in range(num_delays):
    
    Delay = delay_vector[d]
     
    data={} 
        
    print("Starting with delay:", Delay)
    
    for trial in range(num_trials):
    
        nodes = num_nodes*[0]

        #Tree variables
        num_in_strings = [str(i) for i in range(max_blocks+1)]

        for i in range(num_nodes):
            nodes[i] = {i:[] for i in num_in_strings[:-1]}

        # nodes[i] is a dictionary of block tree for node i which contains children of the blocks 0,1,....,T-1    

        god_node_children = {i:[] for i in num_in_strings}    
        god_node_parents = {i:"" for i in num_in_strings[:-1]}

        rechability_matrix= np.zeros((num_nodes,max_blocks+1))
        rechability_matrix[:,0]=1

        delay_variables = np.random.exponential(Delay,(num_nodes,max_blocks+1))



        #Block arrival process
        for t in range(1, max_blocks+1):

            chosen_node = np.random.randint(num_nodes)
            
            delay_variables[chosen_node,t] = 0  # Removing GOD model\n"
            
            # Update block trees for all nodes "n"

            if t==1:
                god_node_children["0"].append(str(t))

                god_node_parents[str(t)] = str(0)


            else:


                #Updating the block trees of all the nodes corresponding to the blocks 1,...t-1

                for tau in range(1,t):

                    for k in range(num_nodes):

                        if delay_variables[k,tau] < t-tau:

                            rechability_matrix[k,tau]=1

                            if str(tau) not in nodes[k][god_node_parents[str(tau)]]:
                                nodes[k][god_node_parents[str(tau)]].append(str(tau))

                #Find the longest path for  chosen node n starting from the genesis block 0,1,,
                
                
                #If we poll we merge the dictionaries a.k.a local block trees and then chose the longest path
                
                #graph = nodes[chosen_node]
                
                
                chosen_graph = nodes[chosen_node]
                
                polling_list=list(range(num_nodes))
                
                polling_list.remove(chosen_node)
                
                polled_node = choice(polling_list)
                
                polled_graph = nodes[polled_node]
                
                graph = {x:latest_union(chosen_graph[x],polled_graph[x],t) for x in chosen_graph}
                
                all_paths = DFS(graph, "0")

                if len(all_paths)==0:

                    chosen_leaf="0"

                else:

                    chosen_path, max_len = give_a_max_path(all_paths)

                    chosen_leaf=chosen_path[-1]


                #Attach this node in the Global tree

                god_node_children[chosen_leaf].append(str(t))

                god_node_parents[str(t)] = chosen_leaf


            _, max_len_tracker[d,t,trial] = give_a_max_path(DFS(god_node_children,"0"))

            
            max_len_tracker[d,t,trial] = max_len_tracker[d,t,trial]-1
            
            
            if t%10 == 0:

                print("Trial {0}: Block {1} arrived : Growth rate of the main chain is {2}".format(trial, t, max_len_tracker[d,t,trial]))
            
            
            
        data["Trial%d"%(trial)]= max_len_tracker[d,:,trial]
        df = pd.DataFrame(data=data)            
        df.to_csv('latest5/'+'latest_3_poll2_n%d_T%d_D%.2f.csv'%(num_nodes,max_blocks,Delay))



Starting with delay: 0.0
Trial 0: Block 10 arrived : Growth rate of the main chain is 10.0
Trial 0: Block 20 arrived : Growth rate of the main chain is 20.0
Trial 0: Block 30 arrived : Growth rate of the main chain is 30.0
Trial 0: Block 40 arrived : Growth rate of the main chain is 40.0
Trial 0: Block 50 arrived : Growth rate of the main chain is 50.0
Trial 0: Block 60 arrived : Growth rate of the main chain is 60.0
Trial 0: Block 70 arrived : Growth rate of the main chain is 70.0
Trial 0: Block 80 arrived : Growth rate of the main chain is 80.0
Trial 0: Block 90 arrived : Growth rate of the main chain is 90.0
Trial 0: Block 100 arrived : Growth rate of the main chain is 100.0
Trial 1: Block 10 arrived : Growth rate of the main chain is 10.0
Trial 1: Block 20 arrived : Growth rate of the main chain is 20.0
Trial 1: Block 30 arrived : Growth rate of the main chain is 30.0
Trial 1: Block 40 arrived : Growth rate of the main chain is 40.0
Trial 1: Block 50 arrived : Growth rate of the ma

Trial 3: Block 40 arrived : Growth rate of the main chain is 40.0
Trial 3: Block 50 arrived : Growth rate of the main chain is 50.0
Trial 3: Block 60 arrived : Growth rate of the main chain is 60.0
Trial 3: Block 70 arrived : Growth rate of the main chain is 70.0
Trial 3: Block 80 arrived : Growth rate of the main chain is 80.0
Trial 3: Block 90 arrived : Growth rate of the main chain is 90.0
Trial 3: Block 100 arrived : Growth rate of the main chain is 100.0
Trial 4: Block 10 arrived : Growth rate of the main chain is 10.0
Trial 4: Block 20 arrived : Growth rate of the main chain is 20.0
Trial 4: Block 30 arrived : Growth rate of the main chain is 30.0
Trial 4: Block 40 arrived : Growth rate of the main chain is 40.0
Trial 4: Block 50 arrived : Growth rate of the main chain is 50.0
Trial 4: Block 60 arrived : Growth rate of the main chain is 60.0
Trial 4: Block 70 arrived : Growth rate of the main chain is 70.0
Trial 4: Block 80 arrived : Growth rate of the main chain is 80.0
Trial 4:

Trial 6: Block 80 arrived : Growth rate of the main chain is 78.0
Trial 6: Block 90 arrived : Growth rate of the main chain is 88.0
Trial 6: Block 100 arrived : Growth rate of the main chain is 98.0
Trial 7: Block 10 arrived : Growth rate of the main chain is 10.0
Trial 7: Block 20 arrived : Growth rate of the main chain is 20.0
Trial 7: Block 30 arrived : Growth rate of the main chain is 30.0
Trial 7: Block 40 arrived : Growth rate of the main chain is 40.0
Trial 7: Block 50 arrived : Growth rate of the main chain is 50.0
Trial 7: Block 60 arrived : Growth rate of the main chain is 60.0
Trial 7: Block 70 arrived : Growth rate of the main chain is 70.0
Trial 7: Block 80 arrived : Growth rate of the main chain is 80.0
Trial 7: Block 90 arrived : Growth rate of the main chain is 90.0
Trial 7: Block 100 arrived : Growth rate of the main chain is 99.0
Trial 8: Block 10 arrived : Growth rate of the main chain is 10.0
Trial 8: Block 20 arrived : Growth rate of the main chain is 18.0
Trial 8:

Trial 1: Block 20 arrived : Growth rate of the main chain is 18.0
Trial 1: Block 30 arrived : Growth rate of the main chain is 27.0
Trial 1: Block 40 arrived : Growth rate of the main chain is 36.0
Trial 1: Block 50 arrived : Growth rate of the main chain is 45.0
Trial 1: Block 60 arrived : Growth rate of the main chain is 54.0
Trial 1: Block 70 arrived : Growth rate of the main chain is 62.0
Trial 1: Block 80 arrived : Growth rate of the main chain is 71.0
Trial 1: Block 90 arrived : Growth rate of the main chain is 80.0
Trial 1: Block 100 arrived : Growth rate of the main chain is 88.0
Trial 2: Block 10 arrived : Growth rate of the main chain is 8.0
Trial 2: Block 20 arrived : Growth rate of the main chain is 15.0
Trial 2: Block 30 arrived : Growth rate of the main chain is 24.0
Trial 2: Block 40 arrived : Growth rate of the main chain is 33.0
Trial 2: Block 50 arrived : Growth rate of the main chain is 42.0
Trial 2: Block 60 arrived : Growth rate of the main chain is 49.0
Trial 2: B

Trial 4: Block 60 arrived : Growth rate of the main chain is 44.0
Trial 4: Block 70 arrived : Growth rate of the main chain is 54.0
Trial 4: Block 80 arrived : Growth rate of the main chain is 63.0
Trial 4: Block 90 arrived : Growth rate of the main chain is 71.0
Trial 4: Block 100 arrived : Growth rate of the main chain is 79.0
Trial 5: Block 10 arrived : Growth rate of the main chain is 9.0
Trial 5: Block 20 arrived : Growth rate of the main chain is 18.0
Trial 5: Block 30 arrived : Growth rate of the main chain is 26.0
Trial 5: Block 40 arrived : Growth rate of the main chain is 33.0
Trial 5: Block 50 arrived : Growth rate of the main chain is 42.0
Trial 5: Block 60 arrived : Growth rate of the main chain is 50.0
Trial 5: Block 70 arrived : Growth rate of the main chain is 58.0
Trial 5: Block 80 arrived : Growth rate of the main chain is 66.0
Trial 5: Block 90 arrived : Growth rate of the main chain is 75.0
Trial 5: Block 100 arrived : Growth rate of the main chain is 84.0
Trial 6: 

Trial 7: Block 100 arrived : Growth rate of the main chain is 78.0
Trial 8: Block 10 arrived : Growth rate of the main chain is 7.0
Trial 8: Block 20 arrived : Growth rate of the main chain is 14.0
Trial 8: Block 30 arrived : Growth rate of the main chain is 22.0
Trial 8: Block 40 arrived : Growth rate of the main chain is 29.0
Trial 8: Block 50 arrived : Growth rate of the main chain is 36.0
Trial 8: Block 60 arrived : Growth rate of the main chain is 44.0
Trial 8: Block 70 arrived : Growth rate of the main chain is 51.0
Trial 8: Block 80 arrived : Growth rate of the main chain is 57.0
Trial 8: Block 90 arrived : Growth rate of the main chain is 65.0
Trial 8: Block 100 arrived : Growth rate of the main chain is 72.0
Starting with delay: 1.75
Trial 0: Block 10 arrived : Growth rate of the main chain is 8.0
Trial 0: Block 20 arrived : Growth rate of the main chain is 14.0
Trial 0: Block 30 arrived : Growth rate of the main chain is 21.0
Trial 0: Block 40 arrived : Growth rate of the mai

Trial 2: Block 40 arrived : Growth rate of the main chain is 28.0
Trial 2: Block 50 arrived : Growth rate of the main chain is 35.0
Trial 2: Block 60 arrived : Growth rate of the main chain is 40.0
Trial 2: Block 70 arrived : Growth rate of the main chain is 46.0
Trial 2: Block 80 arrived : Growth rate of the main chain is 52.0
Trial 2: Block 90 arrived : Growth rate of the main chain is 59.0
Trial 2: Block 100 arrived : Growth rate of the main chain is 65.0
Trial 3: Block 10 arrived : Growth rate of the main chain is 7.0
Trial 3: Block 20 arrived : Growth rate of the main chain is 12.0
Trial 3: Block 30 arrived : Growth rate of the main chain is 19.0
Trial 3: Block 40 arrived : Growth rate of the main chain is 24.0
Trial 3: Block 50 arrived : Growth rate of the main chain is 31.0
Trial 3: Block 60 arrived : Growth rate of the main chain is 37.0
Trial 3: Block 70 arrived : Growth rate of the main chain is 43.0
Trial 3: Block 80 arrived : Growth rate of the main chain is 48.0
Trial 3: B